In [1]:
# Core
import os
import shutil
import xml.etree.ElementTree as ET

# YOLO
from ultralytics import YOLO

# Data handling / visualization
import matplotlib.pyplot as plt
import cv2
import yaml
import random
import pandas as pd
import matplotlib.image as mpimg
import glob
import numpy as np
import seaborn as sns
from scipy.stats import pearsonr
import random



In [2]:
# --------------------------
# 1. Setup device
# --------------------------
device = 'cpu'
print(f"Using device: {device}")

# --------------------------
# 2. Load pre-trained YOLOv8 model
# --------------------------
# Nano YOLOv8 for CPU
model = YOLO("yolov8n.pt")
print("Model loaded:", model)


Using device: cpu
Model loaded: YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.

In [3]:
# --------------------------
# 3. Dataset setup
# --------------------------
# Paths
repo_root = "C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc"
voc_root = os.path.join(repo_root, "VOCdevkit")
yolo_dataset_dir = os.path.join(repo_root, "YOLO_VOC")
os.makedirs(yolo_dataset_dir, exist_ok=True)

# VOC Classes
VOC_CLASSES = [
    'aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
    'diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor'
]

# Function to convert bounding boxes
def convert_bbox(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return x*dw, y*dh, w*dw, h*dh

# Conversion function
def voc_to_yolo(voc_root, year="2012", split="train", output_dir="YOLO_VOC"):
    img_dir = os.path.join(voc_root, f"VOC{year}", "JPEGImages")
    ann_dir = os.path.join(voc_root, f"VOC{year}", "Annotations")
    split_file = os.path.join(voc_root, f"VOC{year}", "ImageSets", "Main", f"{split}.txt")

    # Output dirs
    txt_output_dir = os.path.join(output_dir, split, "labels")
    img_output_dir = os.path.join(output_dir, split, "images")
    os.makedirs(txt_output_dir, exist_ok=True)
    os.makedirs(img_output_dir, exist_ok=True)

    # Read official split list
    with open(split_file, "r") as f:
        img_ids = [line.strip() for line in f.readlines()]

    for img_id in img_ids:
        # Copy image
        src_img = os.path.join(img_dir, f"{img_id}.jpg")
        dst_img = os.path.join(img_output_dir, f"{img_id}.jpg")
        shutil.copy(src_img, dst_img)

        # Convert annotation
        xml_file = os.path.join(ann_dir, f"{img_id}.xml")
        tree = ET.parse(xml_file)
        root = tree.getroot()
        w = int(root.find("size/width").text)
        h = int(root.find("size/height").text)

        yolo_labels = []
        for obj in root.findall("object"):
            cls_name = obj.find("name").text
            if cls_name not in VOC_CLASSES:
                continue
            cls_id = VOC_CLASSES.index(cls_name)
            bbox = obj.find("bndbox")
            xmin = float(bbox.find("xmin").text)
            ymin = float(bbox.find("ymin").text)
            xmax = float(bbox.find("xmax").text)
            ymax = float(bbox.find("ymax").text)
            x_center, y_center, bw, bh = convert_bbox((w, h), (xmin, xmax, ymin, ymax))
            yolo_labels.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

        txt_file_path = os.path.join(txt_output_dir, f"{img_id}.txt")
        with open(txt_file_path, "w") as f:
            f.write("\n".join(yolo_labels))


# Convert train and val sets
voc_to_yolo(voc_root, "2012", "train", yolo_dataset_dir)
voc_to_yolo(voc_root, "2012", "val", yolo_dataset_dir)

# Generate YAML config for YOLOv8
voc_yaml = {
    'train': os.path.join(yolo_dataset_dir, 'train', 'images'),
    'val': os.path.join(yolo_dataset_dir, 'val', 'images'),
    'nc': len(VOC_CLASSES),
    'names': VOC_CLASSES
}

yaml_path = os.path.join(repo_root, "voc.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(voc_yaml, f)
print("YOLO dataset prepared and voc.yaml created at:", yaml_path)

YOLO dataset prepared and voc.yaml created at: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\voc.yaml


In [4]:
# --------------------------
# 3. Subset setup
# --------------------------
# --------------------------
# Paths
# --------------------------
repo_root = "C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc"
voc_root = os.path.join(repo_root, "VOCdevkit", "VOC2012")
subset_dir = os.path.join(repo_root, "YOLO_VOC_subset")
os.makedirs(subset_dir, exist_ok=True)

subset_sizes = {
    "train": 800,   # number of images for CPU testing
    "val": 200
}

# --------------------------
# Function to create subset
# --------------------------
def create_yolo_subset(voc_root, original_dir, subset_dir, split, num_images):
    split_file = os.path.join(voc_root, "ImageSets", "Main", f"{split}.txt")
    orig_images_dir = os.path.join(original_dir, split, "images")
    orig_labels_dir = os.path.join(original_dir, split, "labels")

    subset_images_dir = os.path.join(subset_dir, split, "images")
    subset_labels_dir = os.path.join(subset_dir, split, "labels")
    os.makedirs(subset_images_dir, exist_ok=True)
    os.makedirs(subset_labels_dir, exist_ok=True)

    # Load official split IDs
    with open(split_file, "r") as f:
        split_ids = [line.strip() + ".jpg" for line in f.readlines()]

    # Keep only those that exist in original_dir
    available_images = [f for f in split_ids if f in os.listdir(orig_images_dir)]

    # Random sample
    sampled_images = random.sample(available_images, min(num_images, len(available_images)))
    
    # Take the first N images instead of random sample
    #sampled_images = available_images[:num_images]

    for img_file in sampled_images:
        # Copy image
        shutil.copy(os.path.join(orig_images_dir, img_file),
                    os.path.join(subset_images_dir, img_file))
        # Copy corresponding label
        label_file = img_file.replace(".jpg", ".txt")
        shutil.copy(os.path.join(orig_labels_dir, label_file),
                    os.path.join(subset_labels_dir, label_file))

    print(f"{split} subset created with {len(sampled_images)} images.")

# --------------------------
# Create subsets
# --------------------------
for split in ["train", "val"]:
    create_yolo_subset(voc_root, yolo_dataset_dir, subset_dir, split, subset_sizes[split])

# --------------------------
# Create subset YAML
# --------------------------
subset_yaml = {
    'train': os.path.join(subset_dir, 'train'),
    'val': os.path.join(subset_dir, 'val'),
    'nc': len(VOC_CLASSES),
    'names': VOC_CLASSES
}

subset_yaml_path = os.path.join(repo_root, "voc_subset.yaml")
with open(subset_yaml_path, "w") as f:
    yaml.dump(subset_yaml, f)
print("Subset YAML created at:", subset_yaml_path)

train subset created with 800 images.
val subset created with 200 images.
Subset YAML created at: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\voc_subset.yaml


In [5]:
# --------------------------
# 4. Fine-tune model (CPU-friendly)
# --------------------------
results = model.train(
    data=subset_yaml_path,      #yaml_path for full dataset; subset_yaml_path for debugging/CPU
    epochs=10,            # lower for CPU
    batch=8,             # small batch for CPU
    imgsz=224,           # smaller image size speeds up CPU training
    device=device,        # CPU
    freeze=10           #freezes first 10 layers (backbone)
)

New https://pypi.org/project/ultralytics/8.3.197 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.191  Python-3.10.18 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\voc_subset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, 

In [7]:
# --------------------------
# 5. Compute full validation metrics
# --------------------------
metrics = model.val(
    data=subset_yaml_path,
    split="val",
    imgsz=320,
    conf=0.1
)

print("Overall mAP@0.5:0.95:", metrics.box.map)
print("Overall Precision:", metrics.box.mp)
print("Overall Recall:", metrics.box.mr)


# --------------------------
# 6. Export per-class metrics into DataFrames
# --------------------------

# mAP DataFrame
df_map = pd.DataFrame({
    "class": VOC_CLASSES,
    "mAP@0.5:0.95": metrics.box.maps
})
df_map.loc["Overall"] = ["Overall", metrics.box.map]

# Precision DataFrame
df_precision = pd.DataFrame({
    "class": VOC_CLASSES,
    "Precision": metrics.box.p
})
df_precision.loc["Overall"] = ["Overall", metrics.box.mp]

# Recall DataFrame
df_recall = pd.DataFrame({
    "class": VOC_CLASSES,
    "Recall": metrics.box.r
})
df_recall.loc["Overall"] = ["Overall", metrics.box.mr]

# F1 DataFrame
f1_per_class = 2 * (metrics.box.p * metrics.box.r) / (metrics.box.p + metrics.box.r + 1e-6)
overall_f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6)
df_f1 = pd.DataFrame({
    "class": VOC_CLASSES,
    "F1": f1_per_class
})
df_f1.loc[len(df_f1)] = ["Overall", overall_f1]

print("\nPer-class mAP:\n", df_map)
print("\nPer-class Precision:\n", df_precision)
print("\nPer-class Recall:\n", df_recall)
print("\nPer-class F1:\n", df_f1)

# Save CSVs in results/YOLO_object_detection_results
object_detection_results_dir = os.path.join(repo_root, "results", "YOLO_object_detection_results")
os.makedirs(object_detection_results_dir, exist_ok=True)

# Save to CSVs
df_map.to_csv(os.path.join(object_detection_results_dir, "yolo_val_map.csv"), index=False)
df_precision.to_csv(os.path.join(object_detection_results_dir, "yolo_val_precision.csv"), index=False)
df_recall.to_csv(os.path.join(object_detection_results_dir, "yolo_val_recall.csv"), index=False)
df_f1.to_csv(os.path.join(object_detection_results_dir, "yolo_val_f1.csv"), index=False)



# --------------------------
# 7. Combined grouped bar chart (Precision, Recall, F1)
# --------------------------
classes = df_precision["class"][:-1]  # exclude "Overall"
x = np.arange(len(classes))  # positions
width = 0.25

plt.figure(figsize=(14, 6))
plt.bar(x - width, df_precision["Precision"][:-1], width=width, label="Precision")
plt.bar(x, df_recall["Recall"][:-1], width=width, label="Recall")
plt.bar(x + width, df_f1["F1"][:-1], width=width, label="F1")

plt.xticks(x, classes, rotation=45, ha="right")
plt.ylabel("Score")
plt.title("YOLOv8 Per-Class Precision, Recall, and F1 (Validation)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(object_detection_results_dir, "yolo_perclass_prf1.png"), bbox_inches="tight")
plt.close()

print("All metrics exported: mAP, Precision, Recall, F1 (CSV + individual + grouped bar charts).")

Ultralytics 8.3.191  Python-3.10.18 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
val: Fast image access  (ping: 0.10.0 ms, read: 306.9156.1 MB/s, size: 145.7 KB)
val: Scanning C:\Users\kamed\Desktop\argonne_K\object_detection_with_pascal_voc\YOLO_VOC_subset\val\labels.cache... 1227 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1227/1227  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 154/154 4.9it/s 31.3s0.2ss
                   all       1227       3381      0.617      0.611      0.619      0.449
             aeroplane         54         93      0.658      0.559      0.592      0.468
               bicycle         56         79      0.581      0.633      0.607      0.456
                  bird         74        144      0.574      0.479      0.525      0.341
                  boat         47         81      0.556      0.531      0.497      0.316
                bottle         76        155      0.615    

In [8]:
# Exclude "Overall" row
perclass_map = df_map["mAP@0.5:0.95"][:-1]
perclass_f1 = df_f1["F1"][:-1]
classes = df_map["class"][:-1]

# Compute correlation
corr, pval = pearsonr(perclass_map, perclass_f1)
print("corr", corr)
print("pval", pval)

plt.figure(figsize=(8, 6))

# Scatter with bigger dots and specific color
sns.scatterplot(
    x=perclass_map,
    y=perclass_f1,
    s=120,                # bigger dots
    color="#009E73"       # custom color
)

# Annotate each class with larger, colored labels
for i, cls in enumerate(classes):
    plt.text(
        perclass_map.iloc[i] + 0.005,
        perclass_f1.iloc[i] + 0.005,
        cls,
        fontsize=8,        # larger font
        color="#009E73"
    )

plt.xlabel("mAP@0.5:0.95")
plt.ylabel("F1 Score")
plt.title(f"YOLOv8 Per-Class mAP vs F1 (Validation)\nPearson r={corr:.2f}, p={pval:.3f}")
plt.grid(True)
plt.tight_layout()

# Save

plt.savefig(os.path.join(object_detection_results_dir, "yolo_map_vs_f1_scatter.png"), bbox_inches="tight")
plt.close()

print("Generated scatterplot: Per-class mAP vs F1 (with correlation)")


corr 0.9229321054127909
pval 6.869658006878054e-09
Generated scatterplot: Per-class mAP vs F1 (with correlation)


In [9]:
# --------------------------
# Compare per-class mAP vs F1
# --------------------------


classes = df_map["class"][:-1]  # exclude "Overall"
x = np.arange(len(classes))
width = 0.35  # bar width

plt.figure(figsize=(14, 6))
plt.bar(x - width/2, df_map["mAP@0.5:0.95"][:-1], width=width, label="mAP@0.5:0.95")
plt.bar(x + width/2, df_f1["F1"][:-1], width=width, label="F1 Score")

plt.xticks(x, classes, rotation=45, ha="right")
plt.ylabel("Score")
plt.title("YOLOv8 Per-Class Comparison: mAP vs F1 (Validation)")
plt.legend()
plt.tight_layout()

# Save
plt.savefig(os.path.join(object_detection_results_dir, "yolo_perclass_map_vs_f1.png"), bbox_inches="tight")
plt.close()

print("Generated grouped bar chart: Per-class mAP vs F1")

Generated grouped bar chart: Per-class mAP vs F1


In [10]:
# --------------------------
# 8. Side-by-side GT vs Predictions (10 random val images)
# --------------------------

val_images_dir = os.path.join(subset_dir, "val", "images")
val_labels_dir = os.path.join(subset_dir, "val", "labels")

# Randomly select 10 val images
all_val_images = glob.glob(os.path.join(val_images_dir, "*.jpg"))
sample_val_images = random.sample(all_val_images, min(10, len(all_val_images)))

comparison_dir = os.path.join(object_detection_results_dir, "comparison_predictions")
os.makedirs(comparison_dir, exist_ok=True)

for idx, img_path in enumerate(sample_val_images, start=1):
    # Load GT labels
    label_path = os.path.join(val_labels_dir, os.path.basename(img_path).replace(".jpg", ".txt"))
    gt_boxes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                cls_id, x, y, w, h = line.strip().split()
                cls_id = int(cls_id)
                gt_boxes.append((VOC_CLASSES[cls_id], float(x), float(y), float(w), float(h)))

    # Run YOLO prediction
    results = model.predict(source=img_path, imgsz=320, conf=0.1, verbose=False)
    res = results[0]

    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # Left: Ground truth (draw bounding boxes manually)
    img = mpimg.imread(img_path)
    axes[0].imshow(img)
    axes[0].set_title("Ground Truth")
    axes[0].axis("off")
    h, w = img.shape[:2]
    for cls_name, x, y, bw, bh in gt_boxes:
        xmin = int((x - bw/2) * w)
        ymin = int((y - bh/2) * h)
        xmax = int((x + bw/2) * w)
        ymax = int((y + bh/2) * h)
        rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                             linewidth=2, edgecolor="lime", facecolor="none")
        axes[0].add_patch(rect)
        axes[0].text(xmin, ymin-5, cls_name, color="lime", fontsize=10, weight="bold")

    # Right: YOLO predictions (ultralytics res.plot())
    img_pred = res.plot()
    axes[1].imshow(img_pred)
    axes[1].set_title("YOLO Prediction")
    axes[1].axis("off")

    plt.tight_layout()
    save_path = os.path.join(comparison_dir, f"comparison_{idx:02d}.png")
    fig.savefig(save_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved GT vs Prediction comparison: {save_path}")

Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\results\YOLO_object_detection_results\comparison_predictions\comparison_01.png
Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\results\YOLO_object_detection_results\comparison_predictions\comparison_02.png
Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\results\YOLO_object_detection_results\comparison_predictions\comparison_03.png
Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\results\YOLO_object_detection_results\comparison_predictions\comparison_04.png
Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\results\YOLO_object_detection_results\comparison_predictions\comparison_05.png
Saved GT vs Prediction comparison: C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc

In [ ]:
# --------------------------
# 9. Comparing YOLO and ResNet's classification performance on Pascal VOC
# --------------------------
# 
# # --------------------------
# Paths to CSV files
# --------------------------

resnet_csv = os.path.join(object_detection_results_dir, "resnet_val_precision_recall_f1.csv")
yolo_precision_csv = os.path.join(object_detection_results_dir, "yolo_val_precision.csv")
yolo_recall_csv = os.path.join(object_detection_results_dir, "yolo_val_recall.csv")
yolo_f1_csv = os.path.join(object_detection_results_dir, "yolo_val_f1.csv")

# --------------------------
# Read CSVs
# --------------------------
df_resnet = pd.read_csv(resnet_csv)
df_yolo_precision = pd.read_csv(yolo_precision_csv)
df_yolo_recall = pd.read_csv(yolo_recall_csv)
df_yolo_f1 = pd.read_csv(yolo_f1_csv)

print("1:",df_yolo_precision.shape)
#print("1:",df_yolo_recall)
#("1:",df_yolo_f1)
# --------------------------
# Clean class names (strip whitespace)
# --------------------------
df_resnet['class'] = df_resnet['class'].str.strip()
df_yolo_precision['class'] = df_yolo_precision['class'].str.strip()
df_yolo_recall['class'] = df_yolo_recall['class'].str.strip()
df_yolo_f1['class'] = df_yolo_f1['class'].str.strip()

# --------------------------
# Filter ResNet rows to exclude Macro/Micro averages
# --------------------------
df_resnet_filtered = df_resnet[~df_resnet['class'].isin(['Macro Avg', 'Micro Avg'])].copy()

# Identify valid classes (non-zero metrics)
valid_classes = df_resnet_filtered[
    (df_resnet_filtered['Precision'] > 0) | 
    (df_resnet_filtered['Recall'] > 0) | 
    (df_resnet_filtered['F1'] > 0)
]['class'].tolist()

# Filter ResNet to only valid classes
df_resnet = df_resnet_filtered[df_resnet_filtered['class'].isin(valid_classes)].reset_index(drop=True)

# Filter YOLO CSVs to match valid classes
df_yolo_precision = df_yolo_precision[df_yolo_precision['class'].isin(valid_classes)].reset_index(drop=True)
df_yolo_recall = df_yolo_recall[df_yolo_recall['class'].isin(valid_classes)].reset_index(drop=True)
df_yolo_f1 = df_yolo_f1[df_yolo_f1['class'].isin(valid_classes)].reset_index(drop=True)

# Merge into a single YOLO DataFrame
df_yolo = pd.DataFrame({"class": valid_classes})
df_yolo = df_yolo.merge(df_yolo_precision[['class', 'Precision']], on='class', how='left')
df_yolo = df_yolo.merge(df_yolo_recall[['class', 'Recall']], on='class', how='left')
df_yolo = df_yolo.merge(df_yolo_f1[['class', 'F1']], on='class', how='left')
df_yolo.fillna(0, inplace=True)

print(df_resnet)
print(df_yolo)




1: (21, 2)
       class  Precision    Recall        F1
0  aeroplane   0.500000  0.250000  0.333333
1    bicycle   1.000000  0.250000  0.400000
2       bird   0.800000  0.500000  0.615385
3        car   0.600000  0.315789  0.413793
4        cat   0.916667  0.687500  0.785714
5      chair   0.538462  0.411765  0.466667
6      horse   1.000000  0.250000  0.400000
7  motorbike   1.000000  0.125000  0.222222
8     person   0.835821  0.910569  0.871595
9  tvmonitor   0.500000  0.571429  0.533333
       class  Precision    Recall        F1
0  aeroplane   0.000000  0.000000  0.000000
1    bicycle   0.000000  0.000000  0.000000
2       bird   0.000000  0.000000  0.000000
3        car   0.062731  0.093407  0.075055
4        cat   0.050725  0.093333  0.065727
5      chair   0.000000  0.000000  0.000000
6      horse   0.000000  0.000000  0.000000
7  motorbike   0.000000  0.000000  0.000000
8     person   0.186667  0.245077  0.211920
9  tvmonitor   0.000000  0.000000  0.000000


In [ ]:

# --------------------------
# Function to create side-by-side bar charts
# --------------------------
def plot_metric_comparison(df_resnet, df_yolo, metric, save_path=None):
    x = np.arange(len(df_resnet))  # positions for classes
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12,6))
    ax.bar(x - width/2, df_resnet[metric], width, label='ResNet', color='skyblue')
    ax.bar(x + width/2, df_yolo[metric], width, label='YOLO', color='orange')
    
    ax.set_xticks(x)
    ax.set_xticklabels(df_resnet['class'], rotation=45, ha='right')
    ax.set_ylabel(metric)
    ax.set_title(f'Per-Class {metric} Comparison: YOLO vs ResNet')
    ax.legend()
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# --------------------------
# Generate bar charts for Precision, Recall, F1
# --------------------------
plot_metric_comparison(df_resnet, df_yolo, "Precision", os.path.join(object_detection_results_dir, "comparison_precision.png"))
plot_metric_comparison(df_resnet, df_yolo, "Recall", os.path.join(object_detection_results_dir, "comparison_recall.png"))
plot_metric_comparison(df_resnet, df_yolo, "F1", os.path.join(object_detection_results_dir, "comparison_f1.png"))



<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

In [ ]:
# --------------------------
# Grouped bar chart with 6 categories
# --------------------------
metrics = ['Precision', 'Recall', 'F1']
x = np.arange(len(df_resnet))
width = 0.12

fig, ax = plt.subplots(figsize=(16,7))

# Color-blind friendly palette (Okabe-Ito)
metric_colors = {
    'Precision': '#0072B2',  # blue
    'Recall': '#009E73',     # green
    'F1': '#E69F00'          # orange
}

handles = []
labels = []

for i, metric in enumerate(metrics):
    # ResNet
    bars_resnet = ax.bar(x + (i-1)*2*width, df_resnet[metric], width,
                         color=metric_colors[metric], alpha=0.9)
    handles.append(bars_resnet)
    labels.append(f"ResNet {metric}")

    # YOLO
    bars_yolo = ax.bar(x + (i-1)*2*width + width, df_yolo[metric], width,
                       color=metric_colors[metric], alpha=0.6, hatch='//', edgecolor='black')
    handles.append(bars_yolo)
    labels.append(f"YOLO {metric}")

# Formatting
ax.set_xticks(x)
ax.set_xticklabels(df_resnet['class'], rotation=45, ha='right', fontsize=10)
ax.set_ylabel("Metric Value")
ax.set_ylim(0, 1.05)
ax.set_title("Per-Class Precision, Recall, F1: YOLO vs ResNet", fontsize=14, pad=15)

# Full legend
ax.legend(handles, labels, fontsize=10, loc='upper right', ncol=2)

plt.tight_layout()
plt.savefig(os.path.join(object_detection_results_dir, "comparison_grouped_metrics_clean.png"), bbox_inches='tight')
plt.show()


<Figure size 1600x700 with 1 Axes>

In [ ]:
# --------------------------
# Scatter plot: Precision vs Recall
# --------------------------
plt.figure(figsize=(10, 8))

# ResNet points (blue circles)
plt.scatter(df_resnet['Precision'], df_resnet['Recall'], 
            color="#CC3311", marker="o", s=100, label="ResNet")

# YOLO points (orange triangles)
plt.scatter(df_yolo_precision['Precision'], df_yolo_recall['Recall'], 
            color="#666666", marker="^", s=100, label="YOLO")

# Add class labels with larger font
for i, cls in enumerate(df_resnet['class']):
    plt.text(df_resnet['Precision'][i] + 0.015, df_resnet['Recall'][i] + 0.015, cls,
             fontsize=10, color="#CC3311")
    plt.text(df_yolo_precision['Precision'][i] + 0.015, df_yolo_recall['Recall'][i] - 0.025, cls,
             fontsize=10, color="#666666")

# Labels and formatting
plt.xlabel("Precision", fontsize=12)
plt.ylabel("Recall", fontsize=12)
plt.title("Precision vs Recall per Class: YOLO vs ResNet", fontsize=14)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)

# Set axis limits with small margins so points don't get cut off
plt.xlim(0.08, 1.02)
plt.ylim(0.08, 1.02)

plt.tight_layout()

# Save figure
save_path = os.path.join(object_detection_results_dir, "precision_recall_scatter.png")
plt.savefig(save_path, bbox_inches="tight")
plt.close()

print(f"Scatter plot saved to {save_path}")

Scatter plot saved to C:/Users/kamed/Desktop/argonne_K/object_detection_with_pascal_voc\precision_recall_scatter.png
